In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from lxml import etree
from collections import defaultdict
from pprint import pprint
from config.assistant import *
from config.site_config import *
from tqdm import tqdm

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

request_headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'),
}

In [2]:
with open(filter_list_company_name_path) as f:
    filter_list_company_name = f.read().splitlines()
with open(filter_list_company_name_part_path) as f:
    filter_list_company_name_part = f.read().splitlines()

with open(filter_list_posting_region_path) as f:
    filter_list_posting_region = f.read().splitlines()
with open(filter_list_posting_name_part_path) as f:
    filter_list_posting_name_part = f.read().splitlines()

In [3]:
# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

2934


### 1. 필터링 - 회사명에 부분적으로 포함되는 단어

In [4]:
for filtering_word in filter_list_company_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['company']:
                if job_list['status'] == 'wait':
                    job_list['status'] = 'close'
                    nCnt += 1

    print(filtering_word, nCnt)

학원 0
스터디 0
교육원 0
교육센터 0
평생교육원 0
필라테스 0
아카데미 0
병원 0
부동산 0
중개 0
건축사 0
의원 0
결혼 0
출판 0
손해사정 0
축구 0
스쿨 0
학교 0
산부인과 0
주얼리 0
부티크 0
복지관 0
스투디오 0
에스씨에이게임 0
렌트카 0
스타트업 0
의료센터 0
웨딩 0
에이치알 0
테라로사 0
성형외과 0
변호사 0
화장품 0
대한상공회의소 0
안과 0
복지재단 0
투어 0
체형교정 0
사단법인 0
투자자문 0
노무법인 0
수영장 1


### 2. 필터링 - 공고명에 부분적으로 포함되는 단어

In [5]:
for filtering_word in filter_list_posting_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
                if job_list['status'] == 'wait':
                    job_list['status'] = 'close'
                    nCnt += 1
                    
    print(filtering_word, nCnt)

영업관리 0
매장관리 0
고객 0
사무보조 0
마케팅 0
마케터 0
경영지원 0
NodeJs 0
Nestjs 0
부동산 0
영업사원 0
자재관리 0
회계 0
재무 0
총무 0
영상의학실 0
유지보수 0
고객응대 0
텔레마케터 0
아웃바운드 0
국비지원 0
화장품 0
디자이너 0
인플루언서 0
MD 0
AMD 0
온라인 0
네일리스트 0
웹디자인 0
기술영업 0
회로설계 0
강사 0
경리 0
사무행정 0
보험설계사 0
유튜브 0
유투브 0
국비무료 0
국비지원 0
수행기사 0
행정보조 0
구축사업 0
골프장 0
영업지원 0
Node.js 0
node.js 0
방송장비 0
물류팀 0
물류관리 0
창고관리 0
정보보안 0
품질경영 0
코스메틱 0
자동차튜닝 0
시각디자인 0
악세사리 0
업무제휴 0
무료 0
영상제작 0
생산관리 0
인사팀장 0
자동차영업 0
생산팀 0
창업센터 0
가맹점 0
웹퍼블리셔 0
품질관리 0
임원비서 0
뷰티 0
해외영업 0
중국어 0
새벽배송 0
센터관리 0
무역업무 0
비서 9
해외구매 0
광고영업 1
기획영업 0
영업기획부 1


In [5]:
# JSON 파일 쓰기
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

### 3. 전체현황 - 회사명에 부분적으로 단어 포함

In [4]:
for filtering_word in filter_list_company_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['company']:
                    nCnt += 1

    print(filtering_word, nCnt)

학원 177
스터디 10
교육원 18
교육센터 7
평생교육원 3
필라테스 5
아카데미 194
병원 38
부동산 36
중개 45
건축사 7
의원 42
결혼 12
출판 8
손해사정 5
축구 2
스쿨 16
학교 64
산부인과 2
주얼리 5
부티크 1
복지관 2
스투디오 1
에스씨에이게임 34
렌트카 1
스타트업 4
의료센터 3
웨딩 6
에이치알 65
테라로사 7
성형외과 18
변호사 2


### 4. 하나씩 확인 - 회사명에 부분적으로 단어 포함

In [48]:
filtering_word = "변호사"
ans_list = []
for company_idx, company_title in enumerate(job_group):
    for job_list in job_group[company_title]:
        if filtering_word in job_list['company']:
            ans_list.append(company_title)
print(filtering_word)
print("검출개수: " + str(len(ans_list)))
ans_list = list(set(ans_list))
print("중복제거: " + str(len(ans_list)))
print("----------------------")
for ans in ans_list:
    print(ans)

변호사
검출개수: 2
중복제거: 1
----------------------
변호사이원후법률사무소


### 5. 전체현황 - 공고명에 부분적으로 단어 포함

In [5]:
for filtering_word in filter_list_posting_name_part:
    nCnt = 0

    for company_idx, company_title in enumerate(job_group):
        for job_list in job_group[company_title]:
            if filtering_word in job_list['title']:
                    nCnt += 1
                    
    print(filtering_word, nCnt)

영업관리 49
매장관리 13
고객 171
사무보조 100
마케팅 1035
마케터 399
경영지원 277
NodeJs 1
Nestjs 1
부동산 75
영업사원 18
자재관리 10
회계 196
재무 82
총무 121
영상의학실 1
유지보수 125
고객응대 4
텔레마케터 1
아웃바운드 11
국비지원 7
화장품 130
디자이너 601
인플루언서 13
MD 413
AMD 91
온라인 510
네일리스트 1
웹디자인 69
기술영업 98
프론트엔드 163
회로설계 11
백엔드 271
강사 199
경리 54
사무행정 10
보험설계사 25
유튜브 47
유투브 1
국비무료 9
국비지원 7
수행기사 2
행정보조 2
구축사업 14
골프장 1
영업지원 77
Node.js 12
node.js 3
방송장비 3
물류팀 14
물류관리 17
창고관리 4
정보보안 32
품질경영 7
코스메틱 31
자동차튜닝 3
시각디자인 10
악세사리 2
업무제휴 3
무료 28
영상제작 13
생산관리 31
인사팀장 2
자동차영업 2
생산팀 11
창업센터 1
가맹점 5


### 6. 일정 기한 지난 공고 삭제

In [4]:
import datetime
import json

# JSON 파일 불러오기
file_path = "config/job_group.json"
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
print(nCnt)
print("--------------")

6162
0
--------------


In [2]:
gap_day = 0
removed_cnt = 0

for company_idx, company_title in enumerate(job_group):
    n_cnt = 0
    tmp_list = []

    for job_list in job_group[company_title]:
        input_date = job_list['input_date']
        date_input = datetime.date(int(input_date[0:4]), int(input_date[5:7]), int(input_date[8:10]))

        today = datetime.date.today().isoformat()
        date_today = datetime.date(int(today[0:4]), int(today[5:7]), int(today[8:10]))

        if (date_today - date_input).days < gap_day:
            tmp_list.append(job_list)
            tmp_list[n_cnt]['title_idx'] = n_cnt
            n_cnt += 1
        else:
            removed_cnt += 1
            print(job_list['company'], job_list['title'])
    job_group[company_title] = tmp_list
print('----------------')
print(removed_cnt)

엘로이랩 [엘로이랩] 인공지능/머신러닝/컴퓨터비전(신입&경력)
엘로이랩 [엘로이랩] 인공지능/머신러닝/컴퓨터비전(CTO)
엘로이랩 [엘로이랩] 인공지능 검사장비 기술영업 (B2B)
엘로이랩 [엘로이랩] 인공지능/머신러닝/스타트업/테스트엔지니어(QA QC)
엘로이랩 [엘로이랩]인공지능/자동화장비/기계설계/PLC제어/스타트업(경력)
엘로이랩 [엘로이랩]인공지능/자동화장비/기계설계/스타트업(경력)
앤씨앤 머신 딥러닝 컴퓨터비전 영상 인식 개발자
앤씨앤 [앤씨앤/넥스트칩] 2022년 9월 연구소, 영업마케팅 각 부문 모집
앤씨앤 임베디드 SW 개발자 (Media Framework)
앤씨앤 ISP영상 화질 튜닝
앤씨앤 Hardware 개발자
앤씨앤 BackEnd 개발자
앤씨앤 DATA 수집 라벨링 Training 전문가
앤씨앤 FrontEnd 개발자
앤씨앤 Android App 개발자
앤씨앤 iOS App 개발자
앤씨앤 Aplication SW 개발자
앤씨앤 [앤씨앤/넥스트칩]2022년 9월 연구소, 영업마케팅 각 부문 신입/경력 모집
스위트케이 자연어처리 및 컴퓨터비전 분야 인공지능(AI) 연구원 모집
스위트케이 인공지능(AI) 연구소 자연어처리 및 컴퓨터비전 분야(신입/경력)
스위트케이 3개월 단기 아르바이트(데이터 사업 문서보조)
아고스비전 임베디드/컴퓨터비전 정규직 채용(경력&신입)
글로벌디펜스 (주)글로벌디펜스 머신 딥러닝 컴퓨터비전 개발자 채용 공고
큐픽스 영상처리, 컴퓨터비전/딥러닝
큐픽스 [신입/경력] 영상처리, 컴퓨터비전/딥러닝 (병역특례 가능)
큐픽스 Web Frontend 개발
큐픽스 [정보보안 담당자] Information Security & Compliance
큐픽스 백엔드 개발
큐픽스 [신입/경력] 백엔드, AWS/ELK/Terraform (병역특례 가능)
큐픽스 [신입/경력] 프론트엔드, Angular/TypeScript, (병역특례 가능)
큐픽스 [신입/경력] Angular/TypeScript, WebGL/OpenGL, 병역특례 가능
큐픽스

주식회사메이드리얼티부동산중개법인 사무실 상가 중개 경력무관 채용(프리랜서)
주식회사메이드리얼티부동산중개법인 사무실 임대 / 건물관리 / 법인영업 직원 모집
공간공인중개사사무소 감각 있는 부동산 중개보조원/공인중개사 채용
공간공인중개사사무소 연신내 부동산 중개보조원/공인중개사 채용
세무법인프라이어 [세무법인 프라이어] 직원 채용합니다 [경력]
세무법인프라이어 [세무법인 프라이어] 직원 채용합니다 [신입 및 경력]
소노연구소 디자인(시각디자인/캐릭터디자인) 채용
소노연구소 디자인(시각디자인/캐릭터디자인) 채용 (경력직 우대)
소노연구소 디자인(시각디자인) 채용 (경력자 우대)
스튜디오랩 (경력) 프론트엔드 개발자를 채용합니다.
스튜디오랩 로봇 S/W 연구 (PM)
스튜디오랩 [경력] 백엔드 개발자를 채용합니다.
스튜디오랩 [경력] 컴퓨터 비전 개발자
문토 [문토] 스타트업 Node.js 개발자를 찾습니다. (경력)
문토 [문토] 스타트업 Flutter 개발자를 찾습니다. (경력)
문토 [문토] Front-End 개발자를 찾습니다. (경력)
문토 [문토] Node.js 개발자를 찾습니다. (경력)
문토 [문토] 백엔드 Tech Lead를 찾습니다 (경력)
세나클소프트 [헬스케어 서비스] Windows Client 개발자
세나클소프트 [IT헬스케어] 모바일 앱 개발(Flutter)
세나클소프트 [헬스케어 서비스] 시스템 엔지니어
세나클소프트 [헬스케어 서비스] 백엔드 개발자
세나클소프트 [헬스케어 서비스] 검색 서비스 개발자
세나클소프트 Backend Server개발 경력직 채용
세나클소프트 [IT헬스케어]Windows Client(C# WPF) 개발 경력직 채용
세나클소프트 모바일앱 개발(Flutter) 경력직 채용
히어로애니미술학원 히어로 애니 만화 미술학원 정직원 & 파트 실기 강사 채용
히어로애니미술학원 히어로 애니 만화 미술학원 정직원 & 파트 실기강사 채용
히어로애니미술학원 히어로 애니 만화 미술학원 CS 고객관리 정직원 채용
히어로애니미술학원 히어로 애니

콘센트릭스서비스코리아 [디지털마케팅] 글로벌 이커머스 운영 프로젝트 정규직 모집
콘센트릭스서비스코리아 [글로벌디지털마케팅] 시니어 리쿠르터 정규직 모집
콘센트릭스서비스코리아 [글로벌디지털마케팅]  콘텐츠모니터링 대시보드 개발관리자
콘센트릭스서비스코리아 [디지털마케팅] 이커머스 초급/중급/고급 개발자 (인턴 포함)
콘센트릭스서비스코리아 [글로벌디지털마케팅] 온라인광고 / 퍼포먼스마케팅 운영자 모집
콘센트릭스서비스코리아 [글로벌디지털마케팅] 빌링/계약관리 지원
콘센트릭스서비스코리아 [글로벌디지털마케팅] 웹사이트 운영 PM 정규직 모집
콘센트릭스서비스코리아 [글로벌디지털마케팅] 컨텐츠 기획자 - FAQ&동영상 기획자 모집
콘센트릭스서비스코리아 [글로벌디지털마케팅] 온라인 광고/퍼포먼스마케팅 운영
콘센트릭스서비스코리아 [글로벌디지털마케팅] 이커머스 초급/중급/고급 개발자 모집 (인턴포함)
콘센트릭스서비스코리아 [글로벌디지털마케팅] UX/UI 디자이너 채용
콘센트릭스서비스코리아 [글로벌디지털마케팅] 웹/이커머스 고도화 담당자 정규직 모집
콘센트릭스서비스코리아 [글로벌디지털마케팅] GA Tech Consultant
콘센트릭스서비스코리아 [글로벌디지털마케팅] 디지털 마케팅 컨설팅
콘센트릭스서비스코리아 [글로벌디지털마케팅] 소셜리스닝/Buzz 분석
콘센트릭스서비스코리아 [글로벌디지털마케팅] 글로벌기업 웹사이트 개인화 프로젝트
헤리트 웹사이트 및 백엔드 서버 개발
이스트시큐리티 [ESTsecurity] 서버 백엔드 개발자 (경력)
엠아이큐브솔루션 솔루션 연구개발자 모집
엠아이큐브솔루션 스마트팩토리 솔루션 연구개발직 모집
엠아이큐브솔루션 S/W 개발자 모집
엠아이큐브솔루션 솔루션 연구개발자 (데이터 분석 & AI 알고리즘, APS) 모집
엠아이큐브솔루션 솔루션 연구개발자 (MES, TC, WEB, AI) 모집
엠아이큐브솔루션 솔루션 연구개발자 APS 모집
티맵모빌리티 Machine Learning Engineer
폴스타헬스케어 웹서비스 개발자 채용 (신입/경력)
에스케이쉴더스 

오라산업관리 [송파방이동]사무보조 채용
남다른컴퍼니 주택 디자인 건축설계사 신입/경력직 채용
비비엠씨 디지털 사이니지 설치 및 유지보수
비비엠씨 디지털 사이니지 설치 및 유지보수  신입 모집
잇츠미의원 (월급225이상)잇츠미의원(피부과)동탄점에서 코디네이터 선생님을 모집합니다.
종합축산 [대소,광혜원,진천,음성,금왕] 음성축산물유통단지 한우부산물 전문 유통팀원 구인
헬스보이짐컴퍼니 업계 가장 빠르면서 바른성장을 보이는 헬스보이짐과 함께 성장하실 동료를 모집합니다
보다에이아이 [현대차그룹 분사 스타트업] 머신비전 (Machine Vision) 개발자 모집
보다에이아이 [현대차그룹 분사 스타트업] 백엔드 (Back-end) 개발자 모집
코디아 AS기사 (내근직) 신입/경력직 채용
테라에너지 데이터시각화 및 모바일게임 분야 디자이너 모집
테라에너지 빅데이터 시각화 솔루션 인공지능(A.I) 고도화 개발자 모집
테라에너지 공유경제플랫폼 서비스 및 데이터시각화 개발자/디자이너 모집
테라에너지 공유경제 및 그룹웨어 데이터시각화 개발자/디자이너 모집
테라에너지 새로운 미래를 만들어갈 신입 개발자 모집
테라에너지 WEB/APP 경력 개발자 모집
중원게임즈 [중원게임즈] 자사 개발 및 서비스 PC MMORPG 기획자 / QA / 서버 프로그래머 모집
하이컴텍 (경력) C#, java 개발자를 모십니다. 석박사 우대합니다.
하이컴텍 (경력/프리랜서) C#, java 개발자를 모십니다. 석박사 우대합니다.
에이치티비욘드 [byb] IoT 플랫폼 앱, 웹 QA 테스터
제이네트웍스 [사무보조]사업장 분사로 인한 인원 충원
하미글로벌 2022년 하미글로벌 각 부문 신입 및 경력 상시 채용
하미글로벌 하미글로벌 넥스트엔진 고객지원팀 신입 팀원 모집
안전대부 자금업무 경력(정규)직 채용 연3000-4000(협의)
안전대부 자금회계팀 자금업무 경력직 채용(연 3300~4000협의)
안전대부 자금회계팀 자금/회계 업무 주임급(경력) 3300-4000만 협의가능
데이터마케팅코리아 데이터 분석가,

한국주택금융공사 2022년도 하반기 한국주택금융공사 직무능력 평가기반 신입직원 채용공고
한국선불카드 서비스개발 신입사원 모집
한국선불카드 서비스개발 경력사원 모집
한국선불카드 영업제휴/기획운영/서비스개발/재무회계 담당자 모집
한국선불카드 2022년 유통사업본부 (영업/제휴) 신입/경력 직원 모집
한국선불카드 한국선불카드㈜ 서비스개발 신입/경력사원 모집
한국선불카드 한국선불카드㈜ 팔라고 서비스 기획운영 담당자 모집
한국선불카드 한국선불카드㈜ 영업제휴/기획운영/서비스개발 담당자 모집
한국선불카드 한국선불카드㈜ 기획운영 담당자 모집
컴파트너스 NAVER 쇼핑 CS 및 업무지원
컴파트너스 [재택근무] 네이버 검색광고 광고주 CS
컴파트너스 [분당] NAVER 쇼핑 CS
컴파트너스 업무지원 및 NAVER 쇼핑 판매자CS
컴파트너스 [부평] 네이버 쇼핑 판매자 CS (정규직)
제니코식품 [무역부/영업지원] 신입 및 경력사원 모집
제니코식품 무역부,영업지원부 신입 및 경력사원 모집
경보제약 경보제약 완제의약품 마케팅 및 영업 채용
경보제약 각 부문별 신입/경력직원 모집
파리크라상 10월 경력, 신입직 모집
파리크라상 10월 경력직 모집
퀴네앤드나겔 [외국계기업] 물류업무 부분 신입 및 경력사원 채용
퀴네앤드나겔 [외국계기업] 경영지원 부문 경력사원 채용
삼원특수지 2022년 ㈜삼원특수지 하반기 채용
에이치엘비제약 HLB제약㈜ 각 부문별 채용
한국미쓰비시전기오토메이션 신입/경력사원 채용
더블유스코프코리아 2022년 하반기 신입 및 경력사원 모집
오비맥주 영업부문 인턴(정규직전환가능) 채용
오비맥주 Tech Management Talent Internship(정규직전환가능)채용
SM삼환기업 2022년 하반기 신입 및 경력사원 채용
온라인팜 [한미약품 관계사]온라인팜 영업/본사/물류센터 채용
온라인팜 [한미약품 관계사]온라인팜 영업/본사/물류센터/JVM 채용
크나우프석고보드 크나우프 석고보드㈜ 각 부문 담당자 채용
삼성증권 리서치시스템운영 파트타이머 모집
NICE페이먼츠 202

옥토스케이프 [메인부스] 캐주얼 유니섹스 브랜드 패션 MD 모집
주식회사엠플레이그라운드 웹디자인/마케터 채용
주식회사엠플레이그라운드 그래픽 디자인/마케터 채용
에버그로우 [반려견Biz] 컨텐츠 기획자
글로벌베딩서비스 홈리빙브랜드 아르페지오베이직 온라인MD / 웹디자이너 / CS팀 신입,경력 채용
란체스터에프엔비 외식업 프랜차이즈 [싸움의고수] 마케팅 인턴 모집 (국민취업제도 일경험 프로그램 : 인턴형)
신세계미디어 [초보가능] 광고영업과 부동산영업 같이 하는 영업직 채용
신세계미디어 재계약률 높은 광고영업과 부동산 중개도 배울수 있는 영업직 채용
안국약품 안국약품(주) 신입 및 경력직 수시채용
파멕스 2022년도 신입/경력채용 모집 요강
드림메이커엔터테인먼트리미티드 (SM엔터계열사)드림메이커 부서별 채용공고
암스트롱인터내셔널코리아 [외국계] 부문별 모집 채용 - 정규직
불가리코리아 [불가리코리아] IT Helpdesk (파견계약직/신입지원가능)
리버밴스 [GIST교원창업기업] 수석/프론트엔드/블록체인&백엔드 개발자 채용
리버밴스 GIST교원창업기업 리버밴스(주) 개발자 채용
리버밴스 리버밴스 개발자 채용 공고
탐스에어서비스 항공세일즈 신입/경력 채용
비전과학바이오메디칼 [경기] 경리/회계/영업지원 신입&경력 채용
리테일앤인사이트 채용 및 인사 담당자(리크루터) 경력직 채용
리테일앤인사이트 경영지원(정산, 총무) 담당자 모집
리테일앤인사이트 [예비유니콘기업]토마토 영업본부 채용(영업/기획/지원/보조)
리테일앤인사이트 Web/App 서비스 기획 설계 & UI/UX 디자이너 채용
리테일앤인사이트 전략기획팀 신입/경력직 모집
조은엔지니어링 엔지니어링회사 경리업무 팀원 모집(신입환영)
조은엔지니어링 엔지니어링회사 경리업무 팀원 모집 (신입환영)
이뮨앱스 (주) 이뮨앱스 부문별  인재 수시 채용(경력 또는 신입)
바이렉스 바이렉스 머신비전 엔지니어/기술영업/구매 신입/경력 채용
비전세무회계 비전세무회계(회계사/세무사사무실) 사원모집
비전세무회계 비전세무회계(회계사/세무사

In [3]:
# JSON 파일 쓰기
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()